In [1]:
import numpy as np
import cv2
import os.path
import pickle
import sklearn.model_selection as model_selection

In [2]:
%reload_ext autoreload
%autoreload 2
import knn_utilities as knn

#### Run for Face Mode

In [24]:
features = np.load(os.path.join('Face_Output', 'features.npy'))
labels = np.load(os.path.join('Face_Output', 'feature_labels.npy'))

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    features, labels, test_size=0.2
)

model_path = 'face_knn_model.clf'

#### Run for Iris Mode

In [17]:
train_directory = 'Iris_Output/Iris_Output_Split_Train'
test_directory = 'Iris_Output/Iris_Output_Split_Test'

x_train = np.load(os.path.join(train_directory, 'X_train.npy'))
y_train = np.load(os.path.join(train_directory, 'y_train.npy'))

x_test = np.load(os.path.join(test_directory, 'X_test.npy'))
y_test = np.load(os.path.join(test_directory, 'y_test.npy'))

model_path = 'iris_knn_model.clf'

#### STEP 1: Train the KNN classifier and save it to disk


In [25]:
# Once the model is trained and saved, you can skip this step next time.
print("Training KNN classifier...")

model = knn.train(
    x_train, 
    y_train,
    n_neighbors=2,
)

pickle.dump(model, open(os.path.join(model_path), 'wb'))

print("Training complete!")

Training KNN classifier...
Training complete!


#### STEP 2: Using the trained classifier, make predictions for unknown images

In [26]:


model = pickle.load(open(os.path.join(model_path), 'rb'))

predicted_labels = knn.predict(x_test, y_test, model=model)

cv2.destroyAllWindows()

Data 1 Predicted Label: S0019_F_63 Actual Label: S0016_M_34
Data 2 Predicted Label: S0015_M_26 Actual Label: S0017_F_60
Data 3 Predicted Label: S0009_M_22 Actual Label: S0001_F_30
Data 4 Predicted Label: S0018_F_43 Actual Label: S0002_M_24
Data 5 Predicted Label: S0004_M_16 Actual Label: S0009_M_22
Data 6 Predicted Label: S0007_F_23 Actual Label: S0007_F_23
Data 7 Predicted Label: S0013_M_34 Actual Label: S0009_M_22
Data 8 Predicted Label: S0009_M_22 Actual Label: S0009_M_22
Data 9 Predicted Label: S0008_F_25 Actual Label: S0008_F_25
Data 10 Predicted Label: S0015_M_26 Actual Label: S0006_F_10
Data 11 Predicted Label: S0008_F_25 Actual Label: S0006_F_10
Data 12 Predicted Label: S0004_M_16 Actual Label: S0021_M_25
Data 13 Predicted Label: S0018_F_43 Actual Label: S0018_F_43
Data 14 Predicted Label: S0015_M_26 Actual Label: S0015_M_26
Data 15 Predicted Label: S0006_F_10 Actual Label: S0005_M_11
Data 16 Predicted Label: S0001_F_30 Actual Label: S0001_F_30
Data 17 Predicted Label: S0015_M_

In [31]:
i = 0
peak_accuracy = 0
while i < 100000:
    i += 1

    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        features, labels, test_size=0.2
    )

    model = knn.train(
        x_train,
        y_train,
        n_neighbors=2,
    )

    predicted_labels, accuracy = knn.predict(x_test, y_test, model=model)

    if (accuracy > peak_accuracy):
        peak_accuracy = accuracy

    if (accuracy > 0.75):
        print('Iteration:', i)
        print('Accuracy:', accuracy)
        break

print('Peak accuracy:', peak_accuracy)

Peak accuracy: 0.7014925373134329
